In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torchvision.datasets as datasets
# from torchvision.datasets import MNIST
from keras.datasets import mnist
from torch.utils.data import DataLoader
import torch.optim as optim

from torchvision import transforms
import torchvision
import torchsummary

from networks.discriminator import PatchDiscriminator

Using TensorFlow backend.
/home/danjiii/Program/ML_practice/data-venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/danjiii/Program/ML_practice/data-venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/danjiii/Program/ML_practice/data-venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
class Mnistdenoizing_dataset(torch.utils.data.Dataset):
    def __init__(self, x_train, y_train):
        self.y_train = y_train[:10000]
        self.x_train = x_train[:10000]
        
    def __getitem__(self, idx):
        return torch.tensor(self.x_train[idx]).type(torch.double), torch.tensor(self.y_train[idx]).type(torch.int32)
    
    def __len__(self):
        return len(self.x_train)

In [3]:
model = PatchDiscriminator(c_dim=1)
torchsummary.summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
      SpectralNorm-1           [-1, 64, 14, 14]               0
         LeakyReLU-2           [-1, 64, 14, 14]               0
      SpectralNorm-3            [-1, 128, 7, 7]               0
       BatchNorm2d-4            [-1, 128, 7, 7]             256
         LeakyReLU-5            [-1, 128, 7, 7]               0
      SpectralNorm-6            [-1, 256, 3, 3]               0
       BatchNorm2d-7            [-1, 256, 3, 3]             512
         LeakyReLU-8            [-1, 256, 3, 3]               0
      SpectralNorm-9            [-1, 512, 2, 2]               0
      BatchNorm2d-10            [-1, 512, 2, 2]           1,024
        LeakyReLU-11            [-1, 512, 2, 2]               0
     SpectralNorm-12              [-1, 1, 1, 1]               0
Total params: 1,792
Trainable params: 1,792
Non-trainable params: 0
-----------------------------------